# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *

In [2]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'manipulateAss'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [3]:
# load the data
fil = '../../data/Rep_vs_Dem_tweets.csv'
schem = StructType([StructField('Party', StringType()),
                    StructField('Handle', StringType()),
                    StructField('Tweet', StringType())])
tweets = spark.read.format('csv').options(header=True).schema(schem).load(fil).repartition('Party')

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [62]:
# talk
print('%d records'%tweets.count())
display(tweets.show(truncate=False))

92489 records
+------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+---------------------------------------------+
|Party                                                                                                                   |Handle                                                                                |Tweet                                        |
+------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+---------------------------------------------+
|investigat…"                                                                                                            |null                                                                                  |null     

None

**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [ ]:
# no need

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [ ]:
# no need, as I set the schema

## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [63]:
tweets.select('*', col('Tweet').like('%@LatinoLeader%').alias('seld')).where(col('seld')==True).show(5, truncate=False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----+
|Party   |Handle       |Tweet                                                                                                                                       |seld|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----+
|Democrat|RepDarrenSoto|RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…|true|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----+



## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [64]:
tweets = tweets.select(when(col('Party').isin(['Democrat', 'Republican']), col('Party')).otherwise('Other').alias('Party'), 'Handle', 'Tweet')

tweets.groupBy(col('Party')).count().show()

+----------+-----+
|     Party|count|
+----------+-----+
|Republican|44392|
|  Democrat|42068|
|     Other| 6029|
+----------+-----+



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [65]:
tweets = tweets.select('Party', 'Handle', trim(substring_index(col('Tweet'), 'https', 1)).alias('Tweet'))\
    .where(col('Tweet').isNotNull() & col('Handle').isNotNull())
tweets.show(truncate=False)

+----------+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party     |Handle                                                          |Tweet                                                                                                                                       |
+----------+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Other     | Florida                                                        |1963. More photos:                                                                                                                          |
|Other     | I will join @alisonstewart on @WNYCMidday to discuss my novel  |“Big Guns.”                                    

## 4. Remove any leading or trailing white space in the tweet column

In [ ]:
# already did that

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [66]:
tweets.withColumnRenamed('Party', 'Dem_Rep_Other').show(truncate=False)

+-------------+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Dem_Rep_Other|Handle                                                          |Tweet                                                                                                                                       |
+-------------+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Other        | Florida                                                        |1963. More photos:                                                                                                                          |
|Other        | I will join @alisonstewart on @WNYCMidday to discuss my novel  |“Big Guns.”                     

## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [67]:
tweets.select(concat_ws('-', col('Party'), col('Handle')).alias('Who'), 'Tweet').show(truncate=False)

+----------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Who                                                                   |Tweet                                                                                                                                       |
+----------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Other- Florida                                                        |1963. More photos:                                                                                                                          |
|Other- I will join @alisonstewart on @WNYCMidday to discuss my novel  |“Big Guns.”                                                             

## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [68]:
def refa(x):
    res = ','.join(re.findall('\#[A-Za-z0-9]*', x))
    return res
refaUDF = udf(lambda x: refa(x), ArrayType(StringType(), True))

In [70]:
# can't get this to match all of the patterns in the tweet because of the unexplained errror :-(
thisPatt = '(\#[A-Za-z0-9]*)'
tweets.select((length(col('Tweet')) - length(array_join(split('Tweet', '#'), ''))).alias('mult'),
              regexp_extract(col('Tweet'), thisPatt, 0).alias('Hashtag'), refaUDF('Tweet').alias('Hashtags'), 'Tweet')\
    .orderBy(col('mult').desc()).show(truncate=False)

Py4JJavaError: An error occurred while calling o765.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 118.0 failed 1 times, most recent failure: Lost task 1.0 in stage 118.0 (TID 1233) (192.168.150.128 executor driver): org.apache.spark.SparkException: 
Bad data in pyspark.daemon's standard output. Invalid port number:
  458961458 (0x1b5b3232)
Python command to execute the daemon was:
  ipython3 -m pyspark.daemon
Check that you don't have any unexpected modules or libraries in
your PYTHONPATH:
  /home/ahowe42/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip:/home/ahowe42/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip:/home/ahowe42/spark-3.1.1-bin-hadoop2.7/jars/spark-core_2.12-3.1.1.jar:/home/ahowe42/anaconda3/bin
Also, check if you have a sitecustomize.py module in your python path,
or in your python installation, that is printing to standard output
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFact

# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [19]:
'''from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc'''

df = spark.read.format('csv').options(header=True, inferSchema=True).load('../../data/hospitalvisits.csv')

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [74]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY") # I have *no idea* why this is needed???
dtFmt = 'yyyy-M-d'
df = df.withColumn('dob', to_date(col('dob'), dtFmt))\
    .withColumn('visit1', to_date(col('visit1'), dtFmt))\
    .withColumn('visit2', to_date(col('visit2'), dtFmt))\
    .withColumn('visit3', to_date(col('visit3'), dtFmt))

df.printSchema()
df.show()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)

+----------+---------+----------+----------+----------+----------+
|first_name|last_name|       dob|    visit1|    visit2|    visit3|
+----------+---------+----------+----------+----------+----------+
|  Mohammed|   Alfasy|1987-04-08|2016-01-07|2017-02-03|2018-03-02|
|     Marcy|Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02|
|     Ginny|   Ginger|1986-07-10|2014-08-07|2015-02-03|2016-03-02|
|     Vijay| Doberson|1988-05-02|2016-01-07|2018-02-03|2018-03-02|
|     Orhan|  Gelicek|1987-05-11|2016-05-07|2017-01-03|2018-09-02|
|     Sarah|    Jones|1956-07-06|2016-04-07|2017-08-03|2018-10-02|
|      John|  Johnson|2017-10-12|2018-01-02|2018-10-03|2018-03-02|
+----------+---------+----------+----------+----------+----------+



## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [75]:
df.select(datediff('visit2', 'visit1').alias('time1'), datediff('visit3', 'visit2').alias('time2'))\
    .select(((col('time1')+col('time2'))/2).alias('avgTime')).show()

+-------+
|avgTime|
+-------+
|  392.5|
|  545.5|
|  286.5|
|  392.5|
|  424.0|
|  454.0|
|   29.5|
+-------+



## 8. Can you calculate the age of each patient?

In [76]:
df.select('dob', ((datediff(current_date(), 'dob'))/365).alias('age')).show()

+----------+------------------+
|       dob|               age|
+----------+------------------+
|1987-04-08| 34.14794520547945|
|1986-04-08| 35.14794520547945|
|1986-07-10|34.893150684931506|
|1988-05-02| 33.07945205479452|
|1987-05-11|34.057534246575344|
|1956-07-06| 64.92328767123287|
|2017-10-12|3.6136986301369864|
+----------+------------------+



## 9. Can you extract the month from the first visit column and call it "Month"?

In [77]:
df.select('visit1', month('visit1').alias('visit1_Mo')).show()

+----------+---------+
|    visit1|visit1_Mo|
+----------+---------+
|2016-01-07|        1|
|2015-01-07|        1|
|2014-08-07|        8|
|2016-01-07|        1|
|2016-05-07|        5|
|2016-04-07|        4|
|2018-01-02|        1|
+----------+---------+



## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [84]:
# load the data
fil = '../../data/supermarket_sales.csv'
schem = StructType([StructField('Invoice ID', StringType()), StructField('Branch', StringType()),
                    StructField('City', StringType()), StructField('Customer type', StringType()),
                    StructField('Gender', StringType()), StructField('Product line', StringType()),
                    StructField('Unit price', FloatType()), StructField('Quantity', IntegerType()),
                    StructField('Tax 5%', FloatType()), StructField('Total', FloatType()),
                    StructField('Date', DateType()), StructField('Time', TimestampType()),
                    StructField('Payment', StringType()), StructField('cogs', FloatType()),
                    StructField('gross margin percentage', FloatType()), StructField('gross income', FloatType()),
                    StructField('Rating', FloatType())])
supers = spark.read.format('csv').options(header=True, dateFormat='MM/dd/yyyy', timestampFormat='HH:mm').schema(schem).load(fil)

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [85]:
supers.printSchema()
display(supers.limit(10).toPandas())

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: float (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: float (nullable = true)
 |-- Total: float (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: float (nullable = true)
 |-- gross margin percentage: float (nullable = true)
 |-- gross income: float (nullable = true)
 |-- Rating: float (nullable = true)



,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.690002,7,26.141500,548.971497,2019-01-05,1970-01-01 13:08:00,Ewallet,522.830017,4.761905,26.141500,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.280000,5,3.820000,80.220001,2019-03-08,1970-01-01 10:29:00,Cash,76.400002,4.761905,3.820000,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.330002,7,16.215500,340.525513,2019-03-03,1970-01-01 13:23:00,Credit card,324.309998,4.761905,16.215500,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.220001,8,23.288000,489.048004,2019-01-27,1970-01-01 20:33:00,Ewallet,465.760010,4.761905,23.288000,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.309998,7,30.208500,634.378479,2019-02-08,1970-01-01 10:37:00,Ewallet,604.169983,4.761905,30.208500,5.3
5,699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.389999,7,29.886499,627.616516,2019-03-25,1970-01-01 18:30:00,Ewallet,597.729980,4.761905,29.886499,4.1
6,355-53-5943,A,Yangon,Member,Female,Electronic accessories,68.839996,6,20.652000,433.691986,2019-02-25,1970-01-01 14:36:00,Ewallet,413.040009,4.761905,20.652000,5.8
7,315-22-5665,C,Naypyitaw,Normal,Female,Home and lifestyle,73.559998,10,36.779999,772.380005,2019-02-24,1970-01-01 11:38:00,Ewallet,735.599976,4.761905,36.779999,8.0
8,665-32-9167,A,Yangon,Member,Female,Health and beauty,36.259998,2,3.626000,76.146004,2019-01-10,1970-01-01 17:15:00,Credit card,72.519997,4.761905,3.626000,7.2
9,692-92-5582,B,Mandalay,Member,Female,Food and beverages,54.840000,3,8.226000,172.746002,2019-02-20,1970-01-01 13:27:00,Credit card,164.520004,4.761905,8.226000,5.9


### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [ ]:
# no need

### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [ ]:
# no need

## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [57]:
'''from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])'''

reviews = spark.read.format('csv').options(header=True, inferSchema=True).load('../../data/reviews.csv')

reviews.printSchema()
reviews.show(6,False)

root
 |-- rating: integer (nullable = true)
 |-- review_txt: string (nullable = true)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good                                                                           |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+-----------------------------------------

## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [58]:
reviews = reviews.select('*', split(regexp_replace(trim(lower('review_txt')), '[^a-z\s]', ''), ' ').alias('words'))
reviews.show(truncate=False)

+------+--------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |words                                                                                              |
+------+--------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |[epic, this, is, the, best, movie, i, have, ever, seen]                                            |
|4     |Pretty good                                                                           |[pretty, good]                                                                                     |
|3     |So so. Casti

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [59]:
reviews.select('rating', 'review_txt').where(array_contains(col('words'), 'epic')).show()

+------+--------------------+
|rating|          review_txt|
+------+--------------------+
|     5|Epic. This is the...|
|     5|The most EPIC mov...|
+------+--------------------+



### That's it! Great Job!

In [86]:
sc.stop()